<a id="1"></a> <br>
## Thêm thư viện và dữ liệu

In [ ]:
# Thêm thư viện
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM, GRU
from keras.layers import Dropout
from sklearn.preprocessing import MinMaxScaler
from keras.models import load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint # Added ModelCheckpoint
import itertools

In [ ]:
# Thêm dữ liệu
data =  pd.read_csv('../content/Techcombank Stock Price History.csv', header=0)

<a id="2"></a> <br>
## Loại bỏ cột không cần thiết

In [ ]:
data.head()

,Date,Price,Open,High,Low,Vol,Change %
0,4/23/2025,26100,25750,26300,25150,18230000,3.78%
1,4/22/2025,25150,25250,25650,23950,29520000,-2.33%
2,4/21/2025,25750,26000,26300,25600,12190000,-0.96%
3,4/18/2025,26000,26500,26500,26000,16890000,0.00%
4,4/17/2025,26000,25600,26000,25550,14260000,0.78%


In [ ]:
print(data.columns)
print(data.shape)

Index(['Date', 'Price', 'Open', 'High', 'Low', 'Vol', 'Change %'], dtype='object')
(124, 7)


In [ ]:
data.iloc[:, 2:-1].describe()

,Open,High,Low,Vol
count,124.000000,124.000000,124.000000,1.240000e+02
mean,25017.338710,25252.822581,24779.435484,1.344758e+07
std,1574.995733,1589.885490,1566.260267,9.405251e+06
min,22300.000000,22550.000000,22200.000000,1.240000e+06
25%,23800.000000,24000.000000,23587.500000,8.040000e+06
50%,24400.000000,24700.000000,24150.000000,1.105500e+07
75%,26212.500000,26462.500000,26000.000000,1.464250e+07
max,28100.000000,28500.000000,27850.000000,5.889000e+07


In [ ]:
data = data.drop(['Date', 'Change %'], axis=1)

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 124 entries, 0 to 123
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   Price   124 non-null    int64
 1   Open    124 non-null    int64
 2   High    124 non-null    int64
 3   Low     124 non-null    int64
 4   Vol     124 non-null    int64
dtypes: int64(5)
memory usage: 5.0 KB


<a id="24"></a> <br>
## Chuẩn hoá dữ liệu

In [ ]:
sc = MinMaxScaler(feature_range=(0, 1))
data_scaled = sc.fit_transform(data)

In [ ]:
data_scaled.shape

(124, 5)

In [ ]:
def create_data(df, n_future, n_past, train_test_split_percentage, validation_split_percentage):
    n_feature = df.shape[1]
    x_data, y_data = [], []

    for i in range(n_past, len(df) - n_future + 1):
        x_data.append(df[i - n_past:i, 0:n_feature])
        y_data.append(df[i + n_future - 1:i + n_future, 0])

    split_training_test_starting_point = int(round(train_test_split_percentage*len(x_data)))
    split_train_validation_starting_point = int(round(split_training_test_starting_point*(1-validation_split_percentage)))

    x_train = x_data[:split_train_validation_starting_point]
    y_train = y_data[:split_train_validation_starting_point]

    x_val = x_data[split_train_validation_starting_point:split_training_test_starting_point]
    y_val =  x_data[split_train_validation_starting_point:split_training_test_starting_point]

    x_test = x_data[split_training_test_starting_point:]
    y_test = y_data[split_training_test_starting_point:]

    return np.array(x_train), np.array(x_test), np.array(x_val), np.array(y_train), np.array(y_test), np.array(y_val)

In [ ]:
X_train, X_test, X_val, y_train, y_test, y_val = create_data(data_scaled, n_future=1, n_past=25, train_test_split_percentage=0.8,
                                               validation_split_percentage = 0)

In [ ]:
print(X_train.shape)
print(X_test.shape)

print(y_train.shape)
print(y_test.shape)

(79, 25, 5)
(20, 25, 5)
(79, 1)
(20, 1)


<a id="3"></a> <br>
## Huấn luyện mô hình LSTM


In [ ]:
regressor = Sequential()
regressor.add(LSTM(units=16, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units=16, return_sequences=False))
regressor.add(Dropout(0.2))
regressor.add(Dense(units=1, activation='linear'))
regressor.compile(optimizer='adam', loss='mse', metrics=[tf.keras.metrics.RootMeanSquaredError()])

regressor.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 25, 16)         │         1,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 25, 16)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 16)             │         2,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,537 (13.82 KB)

 Trainable params: 3,537 (13.82 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
#Kiểm tra giá trị RMSE của mô hình LSTM thông thường trước khi điều chỉnh siêu tham số
results = regressor.evaluate(X_test, y_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 755ms/step - loss: 0.0365 - root_mean_squared_error: 0.1910


<a id="4"></a> <br>
## Điều chỉnh siêu tham số LSTM

In [ ]:
def LSTM_HyperParameter_Tuning(config, x_train, y_train, x_test, y_test):

    n_neurons, n_batch_size, dropout = config
    possible_combinations = list(itertools.product(n_neurons, n_batch_size, dropout))

    print(possible_combinations)
    print('\n')

    hist = []

    for i in range(0, len(possible_combinations)):

        print(f'{i+1}th combination: \n')
        print('--------------------------------------------------------------------')

        n_neurons, n_batch_size, dropout = possible_combinations[i]

        regressor = Sequential()
        regressor.add(LSTM(units=n_neurons, return_sequences=True, input_shape=(x_train.shape[1], x_train.shape[2])))
        regressor.add(Dropout(dropout))

        regressor.add(LSTM(units=n_neurons, return_sequences=False))
        regressor.add(Dropout(dropout))
        regressor.add(Dense(units=1, activation='linear'))
        regressor.compile(optimizer='adam', loss='mse', metrics=[tf.keras.metrics.RootMeanSquaredError()])

        es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)

        file_path = 'best_model.h5'

        mc = ModelCheckpoint(file_path, monitor='val_loss', mode='min', verbose=1, save_best_only=True)

        regressor.fit(x_train, y_train, validation_split=0.3, epochs=40, batch_size=n_batch_size, callbacks=[es, mc], verbose=0)

        train_accuracy = regressor.evaluate(x_train, y_train, verbose=0)
        test_accuracy = regressor.evaluate(x_test, y_test, verbose=0)

        hist.append(list((n_neurons, n_batch_size, dropout,
                          train_accuracy, test_accuracy)))

        print(f'{str(i)}-th combination = {possible_combinations[i]} \n train accuracy: {train_accuracy} and test accuracy: {test_accuracy}')

        print('--------------------------------------------------------------------')
        print('--------------------------------------------------------------------')
        print('--------------------------------------------------------------------')
        print('--------------------------------------------------------------------')

    return hist

In [ ]:
config = [[16, 32], [8, 16, 32], [0.2]]

hist = LSTM_HyperParameter_Tuning(config, X_train, y_train, X_test, y_test)

[(16, 8, 0.2), (16, 16, 0.2), (16, 32, 0.2), (32, 8, 0.2), (32, 16, 0.2), (32, 32, 0.2)]


1th combination: 

--------------------------------------------------------------------


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)



Epoch 1: val_loss improved from inf to 0.01143, saving model to best_model.h5



Epoch 2: val_loss did not improve from 0.01143

Epoch 3: val_loss did not improve from 0.01143

Epoch 4: val_loss did not improve from 0.01143

Epoch 5: val_loss did not improve from 0.01143

Epoch 6: val_loss improved from 0.01143 to 0.00914, saving model to best_model.h5



Epoch 7: val_loss did not improve from 0.00914

Epoch 8: val_loss did not improve from 0.00914

Epoch 9: val_loss did not improve from 0.00914

Epoch 10: val_loss improved from 0.00914 to 0.00881, saving model to best_model.h5


...

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)



Epoch 1: val_loss improved from inf to 0.02849, saving model to best_model.h5



Epoch 2: val_loss improved from 0.02849 to 0.00868, saving model to best_model.h5



Epoch 3: val_loss improved from 0.00868 to 0.00391, saving model to best_model.h5



Epoch 4: val_loss did not improve from 0.00391

Epoch 5: val_loss did not improve from 0.00391

Epoch 6: val_loss did not improve from 0.00391

Epoch 7: val_loss did not improve from 0.00391

Epoch 8: val_loss did not improve from 0.00391
Epoch 8: early stopping
5-th combination = (32, 32, 0.2) 
 train accuracy: [0.012830052524805069, 0.11326982080936432] and test accuracy: [0.011545626446604729, 0.10745057463645935]
--------------------------------------------------------------------
--------------------------------------------------------------------
--------------------------------------------------------------------
--------------------------------------------------------------------


<a id="44"></a> <br>
## Chọn mô hình tốt nhất

In [ ]:
hist = pd.DataFrame(hist)
hist = hist.sort_values(by=[4], ascending=True)
hist



,0,1,2,3,4
0,16,8,0.2,"[0.007247017230838537, 0.08512941747903824]","[0.007919518277049065, 0.0889916718006134]"
4,32,16,0.2,"[0.004876057617366314, 0.06982877105474472]","[0.008704757317900658, 0.09329929202795029]"
3,32,8,0.2,"[0.011263368651270866, 0.10612902045249939]","[0.00978812389075756, 0.09893494844436646]"
5,32,32,0.2,"[0.012830052524805069, 0.11326982080936432]","[0.011545626446604729, 0.10745057463645935]"
1,16,16,0.2,"[0.005346021149307489, 0.0731164887547493]","[0.011707378551363945, 0.10820063948631287]"
2,16,32,0.2,"[0.030113346874713898, 0.17353197932243347]","[0.03053266368806362, 0.17473597824573517]"


In [ ]:
print(f'Best Combination: \n n_neurons = {hist.iloc[0, 0]}\n n_batch_size = {hist.iloc[0, 1]}\n dropout = {hist.iloc[0, 2]}')
print('**************************')
print(f'Results Before Tunning:\n Test Set RMSE: {np.round(results, 4)[1]}\n')
print(f'Results After Tunning:\n Test Set RMSE: {np.round(hist.iloc[0, -1], 4)[1]}\n')
print(f'{np.round((results[1] - hist.iloc[0, -1][1])*100/np.round(results, 4)[1])}% Improvement')

Best Combination: 
 n_neurons = 16
 n_batch_size = 8
 dropout = 0.2
**************************
Results Before Tunning:
 Test Set RMSE: 0.191

Results After Tunning:
 Test Set RMSE: 0.089

53.0% Improvement
